In [1]:
from sklearn.model_selection import KFold
import random
import numpy as np
from sklearn.preprocessing import StandardScaler
import function
import matplotlib.pyplot as plt
import math
import tqdm
import optuna

## 条件１


In [ ]:
sJIVE_tes_lis=[]
proposed_tes_lis=[]

#生成するデータの条件
p_1=50
p_2=50
n=150
r_J_or=10
r_1_or=10
r_2_or=10
r_prop=1.0
w_J=1.0
w_1=1.0
w_2=1.0
X1_erro=0.2
X2_erro=0.2
y_erro=0.0001

#sJIVEとproposed共通の条件
k = 5
kf = KFold(n_splits=k)

times=100
threshold=100.0

times_tes=100
threshold_tes=10.0 

#sJIVEに固有の条件


#proposedに固有の条件



for i in range(1):
    #データの生成
    X_1_al,X_2_al,y_al=function.gene_data(p_1,p_2,n,r_J_or,r_1_or,r_2_or,r_prop,w_J,w_1,w_2,X1_erro,X2_erro,y_erro)
    X_1_original=X_1_al[:,:100]
    X_2_original=X_2_al[:,:100]
    y_original=y_al[:,:100]

    X_1_test=X_1_al[:,100:150]
    X_2_test=X_2_al[:,100:150]
    y_test=y_al[:,100:150]

    df_tra=np.row_stack((X_1_original,X_2_original,y_original))
    df_tes=np.row_stack((X_1_test,X_2_test,y_test))

    #sJIVEの実装
    ##パラメータ推定
    ###sJIVEのパラメータ推定の定義式
    def objective(trial):
        #ハイパーパラメータを定義
        eta=trial.suggest_float("eta",0.01,0.99)
        r_J=trial.suggest_int("r_J",1,20)
        r_1=trial.suggest_int("r_1",1,20)
        r_2=trial.suggest_int("r_2",1,20)

        accuracy_sJIVE_cv=function.cv_sJIVE(df_tra,eta=eta,r_J=r_J,r_1=r_1,r_2=r_2,p_1=p_1,p_2=p_2,times=times,threshold=threshold,times_tes=times_tes,threshold_tes=threshold_tes)

        return accuracy_sJIVE_cv
    ###推定
    study=optuna.create_study(direction="minimize")
    study.optimize(objective,n_trials=100)
    best_params = study.best_params
    best_accuracy = study.best_value

    ##sJIVEの実行
    eta=best_params["eta"]
    r_J=best_params["r_J"]
    r_1=best_params["r_1"]
    r_2=best_params["r_2"]

    erro_lis,number_best,erro_best,S_J_best,U_1_best,U_2_best,theta_1_best,S_1_best,W_1_best,theta_21_best,S_2_best,W_2_best,theta_22_best,hat_X_y_best=function.sJIVE(
        eta=eta,times=times,r_J=r_J,r_1=r_1,r_2=r_2,X_1_or=X_1_original,X_2_or=X_2_original,y_or=y_original,threshold=threshold)
    
    ##テストデータに対して実行
    erro_tes_lis,erro_tes_best,S_J_new_best,S_1_new_best,S_2_new_best,y_new,erro_result=function.sJIVE_prediction(
        X_1_test,X_2_test,y_test,U_1_best,U_2_best,W_1_best,W_2_best,theta_1_best,theta_21_best,theta_22_best,times_tes,threshold_tes)
    
    ##結果をリストに代入
    sJIVE_tes_score=np.linalg.norm(y_test-y_new)**2
    sJIVE_tes_lis.append(sJIVE_tes_score)

    #proposed_methodの実行
    ##パラメータ推定
    def objective_2(trial):
        eta=trial.suggest_float("eta",0.01,0.99)
        r_J=trial.suggest_int("r_J",1,20)
        r_1=trial.suggest_int("r_1",1,20)
        r_2=trial.suggest_int("r_2",1,20)
        gamma_1=trial.suggest_float("gamma_1",0.1,10.0)
        gamma_21=trial.suggest_float("gamma_21",0.1,10.0)
        gamma_22=trial.suggest_float("gamma_22",0.1,10.0)

        accuracy_pcLasso_sJIVE_cv=function.cv_pcLasso_sJIVE(df_tra,eta=eta,r_J=r_J,r_1=r_1,r_2=r_2,gamma_1=gamma_1,gamma_21=gamma_21,gamma_22=gamma_22,p_1=p_1,p_2=p_2,times=times,threshold=threshold,times_tes=times_tes,threshold_tes=threshold_tes)

        return accuracy_pcLasso_sJIVE_cv
    
    study_2=optuna.create_study(direction="minimize")
    study_2.optimize(objective_2,n_trials=100)
    best_params_2 = study_2.best_params
    best_accuracy_2 = study_2.best_value

    eta_2=best_params_2["eta"]
    r_J_2=best_params_2["r_J"]
    r_1_2=best_params_2["r_1"]
    r_2_2=best_params_2["r_2"]
    gamma_1=best_params_2["gamma_1"]
    gamma_21=best_params_2["gamma_21"]
    gamma_22=best_params_2["gamma_22"]

    ##proposed_methodを実行
    erro_lis_2,number_bes_2,erro_best_2,S_J_best_2,U_1_best_2,U_2_best_2,theta_1_best_2,S_1_best_2,W_1_best_2,theta_21_best_2,S_2_best_2,W_2_best_2,theta_22_best_2,hat_X_y_best_2=function.pcLasso_sJIVE(
        eta=eta_2,times=times,r_J=r_J_2,r_1=r_1_2,r_2=r_2_2,gamma_1=gamma_1,gamma_21=gamma_21,gamma_22=gamma_22,X_1_or=X_1_original,X_2_or=X_2_original,y_or=y_original,threshold=threshold)
    
    ##テストデータに対して実行
    erro_tes_lis_2,erro_tes_best_2,S_J_new_best_2,S_1_new_best_2,S_2_new_best_2,y_new_2,erro_result_2=function.pcLasso_sJIVE_prediction(
        X_1_test,X_2_test,y_test,U_1_best_2,U_2_best_2,W_1_best_2,W_2_best_2,theta_1_best_2,theta_21_best_2,theta_22_best_2,times_tes,threshold_tes)
    
    ##結果をリストに代入
    pcLasso_sJIVE_tes_score=np.linalg.norm(y_test-y_new_2)**2
    proposed_tes_lis.append(pcLasso_sJIVE_tes_score)




In [ ]:
sJIVE_tes_lis

In [ ]:
proposed_tes_lis

In [ ]:
y_test

In [ ]:
y_new

In [ ]:
np.linalg.norm(y_test-y_new)**2

In [ ]:
y_new_2

# 条件２

In [39]:
sJIVE_tes_lis=[]
proposed_tes_lis=[]

#生成するデータの条件
p_1=50
p_2=50
n=150
r_J_or=5
r_1_or=5
r_2_or=5
r_prop=1.0
W_J_lis=[100,10,0.1,0.1,0.01]
W_1_lis=[100,10,0.1,0.1,0.01]
W_2_lis=[100,10,0.1,0.1,0.01]
X1_erro=0.001
X2_erro=0.001
y_erro=0.001

#sJIVEとproposed共通の条件
k = 5
kf = KFold(n_splits=k)

times=100
threshold=10.0

times_tes=100
threshold_tes=10.0 

#sJIVEに固有の条件


#proposedに固有の条件



for i in range(1):
    #データの生成
    X_1_al,X_2_al,y_al=function.gene_data_2(p_1,p_2,n,r_J_or,r_1_or,r_2_or,r_prop,W_J_lis,W_1_lis,W_2_lis,X1_erro,X2_erro,y_erro)
    X_1_original=X_1_al[:,:100]
    X_2_original=X_2_al[:,:100]
    y_original=y_al[:,:100]

    X_1_test=X_1_al[:,100:150]
    X_2_test=X_2_al[:,100:150]
    y_test=y_al[:,100:150]

    df_tra=np.row_stack((X_1_original,X_2_original,y_original))
    df_tes=np.row_stack((X_1_test,X_2_test,y_test))

    #sJIVEの実装
    ##パラメータ推定
    ###sJIVEのパラメータ推定の定義式
    def objective(trial):
        #ハイパーパラメータを定義
        eta=trial.suggest_float("eta",0.01,0.99)
        r_J=trial.suggest_int("r_J",1,20)
        r_1=trial.suggest_int("r_1",1,20)
        r_2=trial.suggest_int("r_2",1,20)

        accuracy_sJIVE_cv=function.cv_sJIVE(df_tra,eta=eta,r_J=r_J,r_1=r_1,r_2=r_2,p_1=p_1,p_2=p_2,times=times,threshold=threshold,times_tes=times_tes,threshold_tes=threshold_tes)

        return accuracy_sJIVE_cv
    ###推定
    study=optuna.create_study(direction="minimize")
    study.optimize(objective,n_trials=10)
    best_params = study.best_params
    best_accuracy = study.best_value

    ##sJIVEの実行
    eta=best_params["eta"]
    r_J=best_params["r_J"]
    r_1=best_params["r_1"]
    r_2=best_params["r_2"]

    erro_lis,number_best,erro_best,S_J_best,U_1_best,U_2_best,theta_1_best,S_1_best,W_1_best,theta_21_best,S_2_best,W_2_best,theta_22_best,hat_X_y_best=function.sJIVE(
        eta=eta,times=times,r_J=r_J,r_1=r_1,r_2=r_2,X_1_or=X_1_original,X_2_or=X_2_original,y_or=y_original,threshold=threshold)
    
    ##テストデータに対して実行
    erro_tes_lis,erro_tes_best,S_J_new_best,S_1_new_best,S_2_new_best,y_new,erro_result=function.sJIVE_prediction(
        X_1_test,X_2_test,y_test,U_1_best,U_2_best,W_1_best,W_2_best,theta_1_best,theta_21_best,theta_22_best,times_tes,threshold_tes)
    
    ##結果をリストに代入
    sJIVE_tes_score=np.linalg.norm(y_test-y_new)**2
    sJIVE_tes_lis.append(sJIVE_tes_score)

    #proposed_methodの実行
    ##パラメータ推定
    def objective_2(trial):
        eta=trial.suggest_float("eta",0.01,0.99)
        r_J=trial.suggest_int("r_J",1,20)
        r_1=trial.suggest_int("r_1",1,20)
        r_2=trial.suggest_int("r_2",1,20)
        gamma_1=trial.suggest_int("gamma_1",1,1000)
        gamma_21=trial.suggest_int("gamma_21",1,1000)
        gamma_22=trial.suggest_int("gamma_22",1,1000)

        accuracy_pcLasso_sJIVE_cv=function.cv_pcLasso_sJIVE(df_tra,eta=eta,r_J=r_J,r_1=r_1,r_2=r_2,gamma_1=gamma_1,gamma_21=gamma_21,gamma_22=gamma_22,p_1=p_1,p_2=p_2,times=times,threshold=threshold,times_tes=times_tes,threshold_tes=threshold_tes)

        return accuracy_pcLasso_sJIVE_cv
    
    study_2=optuna.create_study(direction="minimize")
    study_2.optimize(objective_2,n_trials=10)
    best_params_2 = study_2.best_params
    best_accuracy_2 = study_2.best_value

    eta_2=best_params_2["eta"]
    r_J_2=best_params_2["r_J"]
    r_1_2=best_params_2["r_1"]
    r_2_2=best_params_2["r_2"]
    gamma_1=best_params_2["gamma_1"]
    gamma_21=best_params_2["gamma_21"]
    gamma_22=best_params_2["gamma_22"]

    ##proposed_methodを実行
    erro_lis_2,number_bes_2,erro_best_2,S_J_best_2,U_1_best_2,U_2_best_2,theta_1_best_2,S_1_best_2,W_1_best_2,theta_21_best_2,S_2_best_2,W_2_best_2,theta_22_best_2,hat_X_y_best_2=function.pcLasso_sJIVE_2(
        eta=0.92,times=times,r_J=r_J_2,r_1=r_1_2,r_2=r_2_2,gamma_1=gamma_1,gamma_21=gamma_21,gamma_22=gamma_22,X_1_or=X_1_original,X_2_or=X_2_original,y_or=y_original,threshold=threshold)
    
    ##テストデータに対して実行
    erro_tes_lis_2,erro_tes_best_2,S_J_new_best_2,S_1_new_best_2,S_2_new_best_2,y_new_2,erro_result_2=function.pcLasso_sJIVE_prediction(
        X_1_test,X_2_test,y_test,U_1_best_2,U_2_best_2,W_1_best_2,W_2_best_2,theta_1_best_2,theta_21_best_2,theta_22_best_2,times_tes,threshold_tes)
    
    ##結果をリストに代入
    pcLasso_sJIVE_tes_score=np.linalg.norm(y_test-y_new_2)**2
    proposed_tes_lis.append(pcLasso_sJIVE_tes_score)

[I 2023-05-02 00:57:28,656] A new study created in memory with name: no-name-a97d49d6-c676-45e1-ae42-097984657faa
100%|██████████| 100/100 [00:00<00:00, 6290.39it/s]
[I 2023-05-02 00:57:35,106] Trial 0 finished with value: 48.236586884395834 and parameters: {'eta': 0.618839845357455, 'r_J': 2, 'r_1': 16, 'r_2': 17}. Best is trial 0 with value: 48.236586884395834.
100%|██████████| 100/100 [00:00<00:00, 8848.74it/s]
[I 2023-05-02 00:57:35,400] Trial 1 finished with value: 37.86398424296486 and parameters: {'eta': 0.08190599017330195, 'r_J': 16, 'r_1': 8, 'r_2': 18}. Best is trial 1 with value: 37.86398424296486.
100%|██████████| 100/100 [00:00<00:00, 7288.48it/s]
[I 2023-05-02 00:57:35,666] Trial 2 finished with value: 41.66803512064574 and parameters: {'eta': 0.6747020242501404, 'r_J': 11, 'r_1': 12, 'r_2': 10}. Best is trial 1 with value: 37.86398424296486.
100%|██████████| 100/100 [00:00<00:00, 3448.84it/s]
[I 2023-05-02 00:57:36,745] Trial 3 finished with value: 48.744876501339995 an

In [40]:
sJIVE_tes_lis

[60.980834887836394]

In [41]:
proposed_tes_lis

[60.94793424722192]

In [42]:
print(best_params_2["eta"])
print(best_params_2["r_J"])
print(best_params_2["r_1"])
print(best_params_2["r_2"])
print(best_params_2["gamma_1"])
print(best_params_2["gamma_21"])
print(best_params_2["gamma_22"])

0.22314580877662063
10
15
10
748
85
220


In [43]:
print(best_params["eta"])
print(best_params["r_J"])
print(best_params["r_1"])
print(best_params["r_2"])

0.08190599017330195
16
8
18


In [44]:
theta_1_best_2

array([[-2.22137118e-04, -2.20536282e-06,  1.14279953e-06,
         6.68762029e-07, -8.01563415e-07,  2.98439781e-07,
        -6.02300477e-09, -5.14460269e-08,  6.20849159e-08,
         8.05943023e-08]])

In [45]:
theta_1_best

array([[-0.22160092, -0.47249043,  0.13941646, -0.20856176, -0.35123759,
         0.20506428, -0.69492198, -0.01324481, -0.04738266, -0.01969445,
        -0.01563702, -0.0346611 , -0.00408795,  0.0121166 , -0.01657412,
         0.02030899]])

In [ ]:
n